# Fidlit.app Development

Welcome to Fidlit.

## Prerequisites

Install free and open source [Podman Desktop](https://podman-desktop.io/) or, if you prefer and own a paid license, [Docker Desktop](https://www.docker.com/products/docker-desktop/).

Next, we will confirm which tool you have installed by asking you for input.

In [57]:
user_container_tool=input("Which tool would you like to use (\"podman\" or \"docker\")? Enter an empty value and it will default to \"podman\".")

if user_container_tool == "":
  user_container_tool = "podman"

valid_container_tools={"podman", "docker"}

if not user_container_tool in valid_container_tools:
    raise ValueError("The value {} is not a valid container tool".format(user_container_tool))

!($user_container_tool --version)

podman version 4.3.1


## Reusable Functions

Here, we'll define a function to run a container using the `podman` command-line to execute commands against the container.

This will allow us to define immutable developer tasks that are consistent across developer machines.

In [58]:
import os

def run_container_with_entrypoint_command_on_volume(tool, tag, name, entrypoint, command, volume, env_file_path):
    """
    Run a container against a mounted volume using a specific entrypoint and command.
    """
    # It is safe to ignore any red "squiggles" when using VS Code on the code immediately below.
    !($tool run \
        --rm \
        --name="$name" \
        --entrypoint="$entrypoint" \
        --volume "$volume":/usr/src/app/ \
        --workdir /usr/src/app/ \
        --env IN_CONTAINER=1 \
        --env-file "$env_file_path" \
        "$tag" \
        "$command")

def run_containerized_task(container_tag, container_entrypoint, task_name):
    """
    Run a containerized task.
    """
    # WARN: use of global variable "user_container_tool"
    run_container_with_entrypoint_command_on_volume(
      user_container_tool,
      container_tag, 
      "fidlit-app-" + task_name,
      container_entrypoint,
      "./" + task_name + ".sh",
      os.getcwd(),
      "./.env")

def run_containerized_node_task(name):
    """
    Run a task in a node container.
    """
    run_containerized_task("localhost/fidlit/node:latest", "/bin/ash", name)

def run_containerized_chrome_task(name):
    """
    Run a task in a chrome container.
    """
    run_containerized_task("localhost/fidlit/node-chrome:latest", "/bin/ash", name)

def run_containerized_cypress_task(name):
    """
    Run a task in a Cypress testing chrome container.
    """
    run_containerized_task("localhost/fidlit/node-cypress-chrome:latest", "/bin/bash", name)

## Initialize

Next, we will initialize the project's dependencies.

In [59]:
run_containerized_node_task("init")

yarn install v1.22.19
[1/4] Resolving packages...
success Already up-to-date.
Done in 0.64s.


## Check

### Linting

Linting checks the styles and formatting of our files and code.

In [60]:
run_containerized_node_task("check-lint")

yarn run v1.22.19
$ ng lint

Linting "app"...

All files pass linting.

Done in 68.20s.


### Unit Tests

Next, we'll run our unit tests.

In [61]:
run_containerized_chrome_task("check-unit")

yarn run v1.22.19
$ ng test
05 02 2023 06:53:15.119:INFO [karma-server]: Karma v6.4.1 server started at http://localhost:9876/
05 02 2023 06:53:15.122:INFO [launcher]: Launching browsers ChromeHeadless with concurrency unlimited
05 02 2023 06:53:15.164:INFO [launcher]: Starting browser ChromeHeadless
05 02 2023 06:53:15.529:INFO [Chrome Headless 109.0.5414.119 (Linux x86_64)]: Connected on socket nsETC1l6WTQmNa0AAAAB with id 32123452
Chrome Headless 109.0.5414.119 (Linux x86_64): Executed 0 of 3 SUCCESS (0 secs / 0 secs)
Chrome Headless 109.0.5414.119 (Linux x86_64): Executed 1 of 3 SUCCESS (0 secs / 0.045 secs)
Chrome Headless 109.0.5414.119 (Linux x86_64): Executed 2 of 3 SUCCESS (0 secs / 0.109 secs)
Chrome Headless 109.0.5414.119 (Linux x86_64): Executed 3 of 3 SUCCESS (0 secs / 0.111 secs)
Chrome Headless 109.0.5414.119 (Linux x86_64): Executed 3 of 3 SUCCESS (0.136 secs / 0.111 secs)
TOTAL: 3 SUCCESS
Done in 92.95s.


- Generating browser application bundles (phase: setup)...
    TypeScript compiler options "target" and "useDefineForClassFields" are set to "ES2022" and "false" respectively by the Angular CLI. To control ECMA version and features use the Browerslist configuration. For more information, see https://angular.io/guide/build#configuring-browser-compatibility
✔ Browser application bundle generation complete.
✔ Browser application bundle generation complete.
✔ Browser application bundle generation complete.


### End-to-end Tests

Now, we'll run our end-to-end (e2e) tests.

In [62]:
run_containerized_cypress_task("check-e2e")

yarn run v1.22.19
$ ng e2e
Passing watch mode to DevServer - watch mode is false

Initial Chunk Files                                                                                     | Names                     |  Raw Size
vendor.js                                                                                               | vendor                    |   3.29 MB |
styles.css, styles.js                                                                                   | styles                    | 360.66 kB |
polyfills.js                                                                                            | polyfills                 | 317.02 kB |
main.js                                                                                                 | main                      |  22.21 kB |
runtime.js                                                                                              | runtime                   |  14.43 kB |

| Initial Total             |   3.99 MB

La

- Generating browser application bundles (phase: setup)...
    TypeScript compiler options "target" and "useDefineForClassFields" are set to "ES2022" and "false" respectively by the Angular CLI. To control ECMA version and features use the Browerslist configuration. For more information, see https://angular.io/guide/build#configuring-browser-compatibility
✔ Browser application bundle generation complete.

For more info see: https://angular.io/guide/build#configuring-commonjs-dependencies

For more info see: https://angular.io/guide/build#configuring-commonjs-dependencies


libva error: vaGetDriverNameByIndex() failed with unknown libva error, driver_name = (null)
[510:0205/065633.978324:ERROR:gpu_memory_buffer_support_x11.cc(44)] dri3 extension not supported.
✔ Browser application bundle generation complete.


## Install

Now, we will install the project.

In [63]:
run_containerized_node_task("install")

yarn run v1.22.19
$ ng build --configuration=production

Initial Chunk Files                   | Names                     |   Raw Size | Estimated Transfer Size
main.3b731bbf32c5f53c.js              | main                      |  484.60 kB |               127.48 kB
styles.08c08b3f8861a443.css           | styles                    |  112.13 kB |                11.96 kB
polyfills.a264a537d11028cf.js         | polyfills                 |   33.08 kB |                10.64 kB
runtime.b770ca7c43cdd92e.js           | runtime                   |    4.59 kB |                 2.21 kB

| Initial Total             |  634.40 kB |               152.29 kB

Lazy Chunk Files                      | Names                     |   Raw Size | Estimated Transfer Size
7701.2cdb6e1b152a4a37.js              | home-home-module          |  132.27 kB |                40.00 kB
2698.acad13668ed58850.js              | swiper-bundle-28080340-js |   97.15 kB |                23.40 kB
polyfills-core-js.c8961a92c3ed4c69

- Generating browser application bundles (phase: setup)...
    TypeScript compiler options "target" and "useDefineForClassFields" are set to "ES2022" and "false" respectively by the Angular CLI. To control ECMA version and features use the Browerslist configuration. For more information, see https://angular.io/guide/build#configuring-browser-compatibility
✔ Browser application bundle generation complete.
✔ Browser application bundle generation complete.
- Copying assets...
✔ Copying assets complete.
- Generating index html...
- Generating index html...
10 rules skipped due to selector errors:
  :host-context([dir=rtl]) .ion-float-start -> subselects_1.subselects[name] is not a function
  :host-context([dir=rtl]) .ion-float-end -> subselects_1.subselects[name] is not a function
  :host-context([dir=rtl]) .ion-float-sm-start -> subselects_1.subselects[name] is not a function
  :host-context([dir=rtl]) .ion-float-sm-end -> subselects_1.subselects[name] is not a function
  :host-context([d